In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)


In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)


In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool


In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()


In [8]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"


In [9]:
sentiment_analysis_tool = SentimentAnalysisTool()


In [10]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)


In [11]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)


In [12]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True,
)


In [13]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To create a detailed profile of DeepLearningAI, I need to gather comprehensive information about the company, including its background, key personnel, recent milestones, and potential needs. I should start by searching the internet to find relevant data about DeepLearningAI, focusing on the required aspects.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profi

In [14]:
from IPython.display import Markdown
Markdown(result)


**Email 1: Subject: Elevating AI Education Together**
Dear Andrew Ng,
I hope this message finds you well. Your dedication to democratizing AI education aligns perfectly with our mission at CrewAI. We have been admiring DeepLearningAI's recent initiatives, particularly your focus on data-centric AI approaches. Our platform offers advanced analytics tools and AI-driven customization that could further enhance DeepLearningAI's content delivery and student engagement. We would be thrilled to explore how we could collaborate to make AI education even more accessible and effective.
Warm regards,
[Your Name]
Lead Sales Representative, CrewAI

**Email 2: Subject: Unlock New Possibilities in AI Learning**
Dear Andrew,
Following our previous conversation, I wanted to highlight how CrewAI's technology can specifically address the needs of DeepLearningAI. Our AI-driven platforms are designed to personalize learning experiences, ensuring that each student can learn at their optimal pace and style. This not only improves learning outcomes but also aligns with your vision of making high-quality AI education universally accessible.
Let's discuss how we can integrate our solutions to revolutionize AI education together.
Best regards,
[Your Name]
Lead Sales Representative, CrewAI

**Email 3: Subject: A Strategic Partnership in AI Education**
Dear Andrew,
It's clear that both DeepLearningAI and CrewAI share a common goal: to transform the landscape of AI education. Our analytics tools can provide invaluable insights into course effectiveness and learner preferences, enabling you to tailor educational content more precisely. Imagine the impact we could have through a strategic partnership that leverages the strengths of both our platforms.
I am looking forward to exploring this potential with you.
Sincerely,
[Your Name]
Lead Sales Representative, CrewAI

**Email 4: Subject: Next Steps Towards Enhancing AI Learning**
Dear Andrew,
Thank you for considering how CrewAI's solutions can play a role in DeepLearningAI's mission. Are you available for a meeting next week to discuss this further? We are excited about the possibility of working together to set new standards in the field of AI education.
Looking forward to your thoughts.
Best wishes,
[Your Name]
Lead Sales Representative, CrewAI